<a href="https://colab.research.google.com/github/aidyai/shoe40k/blob/main/src/eda/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import os
import json
import random
from PIL import Image
import pandas as pd
import wandb

import numpy as np
from pathlib import Path
from tqdm import tqdm

from shutil import copyfile

In [ ]:
# Login with your autorization key
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!wget "https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/FOOT40k.tar.gz?download=true"
!wget "https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/foot40k.json?download=true"

--2024-02-27 23:01:50--  https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/FOOT40k.tar.gz?download=true
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.12, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/21/c6/21c6a561f6c396c222a1d2b05fd063e87a278bb0668e9caa4513f74d34653686/db52b59949c5c5f8a4b2c2365662ee8125d96b5c256fd5c2c30fb9259321cfe7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27FOOT40k.tar.gz%3B+filename%3D%22FOOT40k.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1709334110&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTMzNDExMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzIxL2M2LzIxYzZhNTYxZjZjMzk2YzIyMmExZDJiMDVmZDA2M2U4N2EyNzhiYjA2NjhlOWNhYTQ1MTNmNzRkMzQ2NTM2ODYvZGI1MmI1OTk0OWM1YzVmOGE0YjJj

In [ ]:
## Get dataset from URL
#Images & #Labels
image_files= "./FOOT40k.tar.gz"
label_files = "./foot40k.json"

In [ ]:
#extract tar files
#!tar -xzvf "./FOOT40k.tar.gz"
print(len(os.listdir("./FOOT40k")))

41659


In [ ]:
# Load JSON data into a DataFrame
df = pd.read_json(label_files)

# Save DataFrame to CSV
df.to_csv('train.csv', index=False)

Conversion completed. CSV file saved as output.csv


In [ ]:
df = pd.read_csv("./train.csv")
df.tail(3)

,file_name,Label
41656,AJ7311-200_Tenis-Nike-Air-Force-1-Hi-Utility-F...,Sneakers
41657,AJ1285-700_Tenis-Nike-Air-Max-90-Essential-Mas...,Sneakers
41658,AJ6910-601_AJ6910_Tenis-Nike-Air-Vapormax-Flyk...,Sneakers


In [ ]:
print(len(df))

In [ ]:
IMG_RES = 512
WANDB_PROJECT = 'lora-vit40k'
RAW_DATA_AT = f'raw_data_{IMG_RES}'

In [ ]:
# Read csv file
df = pd.read_csv('./train.csv')

# Integer encode multi-label labels.
labels = list(df['Label'].value_counts().keys())
labels_dict = dict(zip(labels, range(12)))
id_to_labels = {val: key for key, val in labels_dict.items()}

In [ ]:
id_to_labels


{0: 'Sneakers',
 1: 'Dressing Shoe',
 2: 'Heels',
 3: 'Sandals',
 4: 'Boot',
 5: 'Crocs'}

In [ ]:
cf = df.replace({"labels": labels_dict})
cf.head(2)

,file_name,Label
0,SHOEDR1175.jpg,Dressing Shoe
1,SHOEDR1177.jpg,Dressing Shoe


In [ ]:
cf.columns

Index(['file_name', 'Label'], dtype='object')

In [ ]:
# Initialize a new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")
# Upload raw csv file as artifact
raw_data_at.add_file('./train.csv')

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label"])


wandb: Currently logged in as: aidysosu. Use `wandb login --relogin` to force relogin


In [ ]:
TRAIN_PATH = "./FOOT40k/"

In [ ]:
label = df.loc[5]['Label']
label

'Dressing Shoe'

In [ ]:
for i in tqdm(range(len(df))):
    image_name = df.loc[i]['file_name']
    label = df.loc[i]['Label']
    full_path = TRAIN_PATH+image_name

    # Append each example as a new row in Table.
    preview_dt.add_data(image_name, wandb.Image(full_path), label)


100%|██████████| 41659/41659 [04:43<00:00, 146.92it/s]


In [ ]:
# save artifact to W&B
raw_data_at.add(preview_dt, "data_split")
run.log_artifact(raw_data_at)
run.finish()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/FOOT40K.csv')
df['Label'] = df['Label'].apply(lambda x: x[0:4]).astype('category').cat.codes

# Split dataset into training and testing sets with stratified sampling
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Label'], random_state=42)


In [ ]:
val_df

In [ ]:
# Load your CSV file containing image filenames and labels
data_df = pd.read_csv('/content/FOOT40K.csv')

In [ ]:
data_df

In [ ]:

# Split dataset into training and testing sets with stratified sampling
train_df, test_df = train_test_split(data_df, test_size=0.2, stratify=data_df['Label'], random_state=42)

# Check the distribution of labels in the training and testing sets
print("Training set label distribution:")
print(train_df['Label'].value_counts(normalize=True))

print("\nTesting set label distribution:")
print(test_df['Label'].value_counts(normalize=True))


In [ ]:
data_df['Label'] = data_df['Label'].apply(lambda x: x[0:4]).astype('category').cat.codes
########
DF = data_df
DF

In [ ]:
data_df['Label'].unique()

array(['Dressing Shoe', 'Boot', 'Crocs', 'Heels', 'Sandals', 'Sneakers'],
      dtype=object)

In [ ]:
'heels': 0,
'sneakers': 1,
'boots': 2,
'sandals': 3,
'crocs': 4,
'dressing shoe': 5,

In [ ]:
DF['Label'].unique()

array([2, 0, 1, 3, 4, 5], dtype=int8)

In [ ]:
len(train_df)

33327

In [ ]:
len(test_df)

8332

In [ ]:
!pip install pytorch_lightning

In [ ]:
#!wget https://huggingface.co/datasets/aidystark/SHOE40K/resolve/main/footnet.tar.gz?download=true
!tar -xvzf /content/footnet.tar.gz -C /content/

In [ ]:
import os
print(len(os.listdir('/content/FOOT40k')))

41659


In [ ]:
!git clone https://github.com/aidyai/shoe40k.git

Cloning into 'shoe40k'...
remote: Enumerating objects: 284, done.
remote: Counting objects: 100% (281/281), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 284 (delta 134), reused 165 (delta 63), pack-reused 3
Receiving objects: 100% (284/284), 115.82 KiB | 2.27 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [ ]:
%cd shoe40k

/content/shoe40k


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python train.py

config.json: 100% 69.7k/69.7k [00:00<00:00, 6.94MB/s]
model.safetensors: 100% 346M/346M [00:04<00:00, 84.2MB/s]
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(len(os.listdir('/content/drive/MyDrive/SHOES/SHOE_DR')))

2379


In [ ]:
#!wget https://huggingface.co/datasets/aidystark/SHOE40K/resolve/main/footnet.tar.gz?download=true
!tar -xvzf /content/footnet.tar.gz -C /content/

import os
print(len(os.listdir('/content/FOOT40k')))

In [ ]:
import os
print(len(os.listdir('/content/FOOT40k')))

41659


In [ ]:
#!unzip /content/footnet.tar.gz
!tar -xvzf /content/footnet.tar.gz -C /content/


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install datasets

In [ ]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Value, Image



#definining image directory
image_dir = "/content/FOOT40k"

#defining path to train and test csv files
train_csv = "/content/FOOT40K.csv"

###-os.path.join(image_dir, "./traindf.csv")
###-test_csv = os.path.join(image_dir, "./testdf.csv")



def load_image(img_path):
    with open(img_path, 'rb') as img:
        return img.read()

def create_dataset(csvFile, split_name):
    df = pd.read_csv(csvFile)
    df['image'] = df['file_name'].map(lambda file_name: load_image(os.path.join(image_dir, file_name)))

    # Reset index to avoid index column
    df = df.reset_index(drop=True)

    # Define features of the dataset
    features = Features({
        'image':Image(),
        'Label':Value('string'),
    })

    dataset = Dataset.from_pandas(df, features=features)
    return dataset

# TRAIN&TEST DATASET CREATION
train_dataset = create_dataset(train_csv, "train")

# Combine into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
})

# Push to hub
dataset_dict.push_to_hub('aidyai/40k')

In [ ]:
## creating a repository
from huggingface_hub import create_repo
from huggingface_hub import HfApi


#create_repo("aidystark/shoe41k", repo_type="dataset")

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/footnet.tar.gz",
    path_in_repo="shoe40k.tar.gz",
    repo_id="aidystark/shoe41k",
    repo_type="dataset",
)



CommitInfo(commit_url='https://huggingface.co/datasets/aidystark/shoe41k/commit/1bfc58fe6525797dc6e530225dde00955d72334d', commit_message='Upload shoe40k.tar.gz with huggingface_hub', commit_description='', oid='1bfc58fe6525797dc6e530225dde00955d72334d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# coding=utf-8
# Copyright 2021 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Dataset class for Shoe40k dataset."""

import datasets
from datasets.tasks import ImageClassification
import pandas as pd
import json
import requests


_HOMEPAGE = "https://huggingface.co/datasets/aidystark/shoe41k"

_DESCRIPTION = (
"----------------------------------------"
)

_CITATION = """\

"""

_LICENSE = """\
LICENSE AGREEMENT
=================
"""

_NAMES      = ['Dressing Shoe', 'Boot', 'Crocs', 'Heels', 'Sandals', 'Sneakers']
_CSV   =  "https://huggingface.co/datasets/aidystark/shoe41k/resolve/main/FOOT40K.csv"
_URL   =  "https://huggingface.co/datasets/aidystark/shoe41k/resolve/main/shoe40k"



df = pd.read_csv(_CSV)
imgLabels = df['Label']


class shoe40k(datasets.GeneratorBasedBuilder):
    """-------"""

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "image": datasets.Image(),
                    "label": datasets.ClassLabel(names=_NAMES),
                }
            ),
            supervised_keys=("image", "label"),
            homepage=_HOMEPAGE,
            citation=_CITATION,
            license=_LICENSE,
            task_templates=[ImageClassification(image_column="image", label_column="label")],
        )

    def _split_generators(self, dl_manager):
        path        = dl_manager.download(_URL)
        image_iters = dl_manager.iter_archive(path)
        return [datasets.SplitGenerator(datasets.Split.TRAIN,gen_kwargs={"images":image_iters,})]


    def _generate_examples(self, images):
        """Generate images and labels for splits."""
        idx = 0
        #Iterate through images
        for filepath,image in images:
            yield idx, {
                "image":{"path":filepath, "bytes":image.read()},
                "label":imgLabels[idx]
            }
            idx += 1


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

data = load_dataset("aidystark/shoe41k")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 